In [ ]:
import os, math, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

from time import sleep

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
config.log_device_placement = True 
sess = tf.Session(config=config)
set_session(sess)


def read_tfrecord(example):
    features = {
        'image': tf.FixedLenFeature([], tf.string),
        'labels': tf.FixedLenFeature([], tf.string)
    }
    sample=tf.parse_single_example(example, features)
    image = tf.decode_raw(sample['image'], tf.uint8)
    image = tf.reshape(image, tf.stack([224, 224, 3]))
    image = augmentation(image)
    labels = tf.decode_raw(sample['labels'], tf.float64)
    labels = tf.reshape(labels, tf.stack([2,19]))
    labels = labels[:,:15] #remove head joints
    labels = tf.cast(labels, tf.float32)
    return image, labels/224.

def load_dataset(filenames):
    files = tf.data.Dataset.list_files(filenames)
    dataset = files.apply(tf.data.experimental.parallel_interleave(tf.data.TFRecordDataset, cycle_length=4))
    dataset = dataset.apply(tf.data.experimental.map_and_batch(map_func=read_tfrecord, batch_size=BATCH_SIZE, drop_remainder=True))
    dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(128, -1))
    dataset = dataset.prefetch(buffer_size=128)
    return dataset

def augmentation(img):
    image = tf.cast(img, tf.float32)
    image = tf.image.random_brightness(image, max_delta=25/255)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.per_image_standardization(image)
    return image

def get_batched_dataset(filenames):
    dataset = load_dataset(filenames)
    return dataset

def get_training_dataset():
    return get_batched_dataset(training_filenames)

def get_validation_dataset():
    return get_batched_dataset(validation_filenames)



def conv(input, kernel, filt, stride):
    x = layers.Conv2D(filters=filt, kernel_size=kernel, strides=stride)(input)
    return x


def resblock(input, filters):
    x = conv(input, 1, filters*4, 1)
    x = layers.BatchNormalization(axis=-1, fused=True)(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
#     x = tf.keras.layers.Activation('relu')(x)    
    x = conv(x, 3, filters, 1)
    x = layers.BatchNormalization(axis=-1, fused=True)(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
#     x = tf.keras.layers.Activation('relu')(x)      
    x = conv(x, 1, filters, 1)
    x = layers.BatchNormalization(axis=-1, fused=True)(x)
    short = conv(input, 3, filters, 1)
    short = layers.BatchNormalization(axis=-1 , fused=True)(short)
    x = layers.Add()([x, short])
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
#     x = tf.keras.layers.Activation('relu')(x)
    return x

def conv_down(input, filters):
    x = conv(input, 3, filters, 2)
    x = layers.BatchNormalization(axis=-1, fused=True)(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
#     x = tf.keras.layers.Activation('relu')(x)    
    return x

def create_model():
    Input = layers.Input(shape=(224, 224, 3))
    x = resblock(Input, 32)
    x = resblock(x, 64)
    ####################
    x = conv_down(x, 64)
    ####################
    x = resblock(x, 64)
    x = resblock(x, 64)
    #####################
    x = conv_down(x, 128)
    #####################
    x = resblock(x, 64)
    x = resblock(x, 64)
    #####################
    x = conv_down(x, 128)
    #####################
    x = resblock(x, 64)
    x = resblock(x, 64)
    #####################
    x = conv_down(x, 128)
    #####################
    x = resblock(x, 64)
    x = resblock(x, 64)
    #####################
    x = resblock(x, 64)
    x = resblock(x, 64)
    #####################
    
     
    x = conv(x, 1, 30, 1)
#     x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    
    
    
    x = tf.keras.layers.Reshape((2,15))(x)
    model = tf.keras.Model(inputs=Input, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(0.01), loss='mean_squared_error')
    return model

model=create_model()

model.summary()

In [ ]:
EPOCHS=200
name='model_grad'
BATCH_SIZE = 8
bucket_train = '/home/antreas/Desktop/tf_records_file/try/*'
bucket_valid = '/home/antreas/Desktop/tf_records_file/try/*'
train = tf.gfile.Glob(bucket_train)
valid = tf.gfile.Glob(bucket_valid)
training_filenames = train
validation_filenames = valid
steps_per_epoch = int(len(train)*1000/(BATCH_SIZE)) ###### to 1000 einai to plithos ton eikonon sto tfrecord #######
val_steps = int(len(valid)*1000/(BATCH_SIZE))


######## Tensorboard ############
logdir='/home/antreas/Desktop/model_feb/'+name
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True)
######## Model Save #############
filepath = '/home/antreas/Desktop/model_feb/'+name+'.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint((filepath),monitor='val_loss', save_best_only=True, verbose=1, save_weights_only=False)
#################################
clbk=[tensorboard_callback, checkpoint]

model=create_model()

model.fit(get_training_dataset(), validation_data=get_validation_dataset(), initial_epoch=0,
          steps_per_epoch=steps_per_epoch ,
          validation_steps=val_steps, epochs=EPOCHS, verbose=1, callbacks=clbk)


In [ ]:
train_images=np.load('/home/antreas/Desktop/test_images/RGB_IM_ARRAY.npy')##### Images

y=train_images[25]
im = np.expand_dims(y, axis=0)
x = model.predict(im)

x[0]

In [ ]:
i = x[0]*224
plt.imshow(y)
plt.plot(i[0][:], i[1][:] , '.' ,color='red')